<a href='http://www.holoviews.org'><img src="assets/hv+bk.png" alt="HV+BK logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>12. Parameters and Widgets</h2></div>

In previous sections we have mostly been learning about using HoloViews to build visualizations. There is only so much information that can be packed on to the screen at once, and in practice it's often necessary to supply interactive widgets so the user can further select what is shown. HoloViews can provide widgets automatically for dimensions that have been declared on the data, but often you will want to express other types of user-settable parameters to control your plots. Here we will discover how to leverage the ``param``, ``paramnb`` and ``parambokeh`` libraries to declare your own custom parameters and how to link widgets to your visualization.

In [ ]:
import param
import paramnb
import parambokeh

import numpy as np
import holoviews as hv
from bokeh.document import Document

hv.extension('bokeh')

# Declaring parameters

As a first step we need to declare a so called ``Parameterized`` class with a number of parameters, which will control our visualization. In this example we will stick with something simple and we will simply expose some styling options for a Curve plot, declaring ``alpha``, ``color``, and ``line_width`` parameters. We'll use a special type of ``Parameterized`` called a ``Stream`` that defines an ``event`` method that we will use later.

In [ ]:
class StyleOptions(hv.streams.Stream):

    alpha = param.Magnitude(default=0.8)
    
    color = param.ObjectSelector(default='red', objects=['red', 'green', 'blue'])
    
    line_width = param.Number(default=1, bounds=(0, 10)) 

## Displaying parameters

We can automatically get a UI to control these parameters using the ``parambokeh`` and ``paramnb`` libraries. Simply call the ``Widgets`` function with a ``Parameterized`` instance as the first argument and it will display a set of widgets based on ``bokeh`` or ``ipywidgets``:

#### ParamBokeh

In [ ]:
opts = StyleOptions()
parambokeh.Widgets(opts)

#### ParamNB

In [ ]:
opts = StyleOptions()
paramnb.Widgets(opts)

## Linking parameters as streams

As a final step we will link the Parameterized class and the widgets to an actual plot. To do this, we will declare a ``DynamicMap``, which sets the style options on an existing ``Curve``. The only thing we need to declare is that the ``Widgets`` should call the ``event`` method on our stream whenever the widget is updated. This will immediately link the widgets to the actual plot.

In [ ]:
opts = StyleOptions()
curve = hv.Curve(np.sin(np.linspace(0, np.pi*3, 100)))
dmap = hv.DynamicMap(lambda **kwargs: curve.opts(style=kwargs), streams=[opts])
paramnb.Widgets(opts, callback=opts.event, continuous_update=True)
dmap

## Laying out plots and widgets

In a Jupyter notebook, the widgets and the plot they control can be in any cell, because the notebook ensures that all of them are laid out on to the screen.  When deploying the dashboard as a standalone bokeh app, we have to combine the plot and widgets into a single layout, because there is no default linear ordering of "cells" at the Bokeh server level. The easiest way to build such a layout is to use the BokehRenderer to convert our plot to a Bokeh model, then pass that to the ``parambokeh.Widgets`` call. We can also supply a ``view_position`` telling it how to lay out the widgets and plots.

In [ ]:
viewer = StyleOptions()
curve = hv.Curve(np.sin(np.linspace(0, np.pi*3, 100)))
dmap = hv.DynamicMap(lambda **kwargs: curve.opts(style=kwargs), streams=[viewer])

# Get bokeh model for the plot
plot = hv.renderer('bokeh').get_plot(dmap, doc=Document()).state

parambokeh.Widgets(viewer, callback=viewer.event, view_position='right',
                   continuous_update=True, plots=[plot])

In [ ]:
## Exercise: Subclass the StyleOptions class and add a parameter to control the line_dash style option

# Hint: The valid options for line_dash are: 'solid', 'dashed', 'dotted', 'dotdash', 'dashdot'

At present, only these relatively simple options for layout are supported, but these objects can be used in an arbitrary Jinja2 template (not shown), and additional Python layout options are expected to be added over time.